<a href="https://colab.research.google.com/github/Wayne122/CMPE257/blob/main/Blastoff_Content_Statistics_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Prerequisite

In [ ]:
%%bash
git clone https://github.com/Wayne122/Content_Statistics_Demo.git
cd Content_Statistics_Demo
pip install -r requirements.txt
python3 -m spacy download en_core_web_lg

  Created wheel for en-core-web-lg: filename=en_core_web_lg-2.2.5-cp36-none-any.whl size=829180944 sha256=474bbe849e47faab750acfa914fe30cda44345e6bcd3739236ea75a16f9cdc64
  Stored in directory: /tmp/pip-ephem-wheel-cache-dm9mce6x/wheels/2a/c1/a6/fc7a877b1efca9bc6a089d6f506f16d3868408f9ff89f8dbfc
Successfully built en-core-web-lg
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


Cloning into 'Content_Statistics_Demo'...


## Import module

In [ ]:
# Restart runtime before running this cell
%cd Content_Statistics_Demo/
from Blastoff_Content_Statistics import BlastoffContentStatistics

/content/Content_Statistics_Demo


/usr/local/lib/python3.6/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


##Loading data

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import pandas as pd

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

downloaded = drive.CreateFile({'id':"1sFMftHLILNfuS0kpTlsbUzWpSNs3WkeD"})
downloaded.GetContentFile('liar_plus_dataset.zip')

In [ ]:
from zipfile import ZipFile

In [ ]:
with ZipFile('liar_plus_dataset.zip', 'r') as myzip:
    train_data = myzip.open('train2.tsv')
    test_data = myzip.open('test2.tsv')
    valid_data = myzip.open('val2.tsv')

train_p_df = pd.read_csv(train_data, sep='\t', header=None).drop([0], axis=1).dropna(how='all')
test_p_df = pd.read_csv(test_data, sep='\t', header=None).drop([0], axis=1).dropna(how='all')
valid_p_df = pd.read_csv(valid_data, sep='\t', header=None).drop([0], axis=1).dropna(how='all')

## Rename the columns

as the module requires the data to have "Statement" and "Label" to train. If you just want to inference, then "Statement" is enough.

In [ ]:
train_p_df.rename(columns={2:'Label', 3:'Statement'}, inplace=True)
test_p_df.rename(columns={2:'Label', 3:'Statement'}, inplace=True)

In [ ]:
X_train = train_p_df.drop('Label', axis=1)
y_train = train_p_df['Label']
X_test = test_p_df.drop('Label', axis=1)
y_test = test_p_df['Label']

## Initialize module

BlastoffContentStatistics(dpos=True, sa=True, ner=True, mpath=None)

*   dpos - Detailed POS tagging
*   as - Sentiment analysis
*   ner - Named-entity recognition
*   mpath - pretrained model path

In [ ]:
bcs = BlastoffContentStatistics()

Loading default pretrained model.


## Extract feature

In [ ]:
bcs.extract(X_train)

Model not found: here.pickle


,CC,CD,DT,EX,FW,IN,JJ,LS,MD,NN,PDT,POS,PRP,RB,RP,SYM,TO,UH,VB,WH,SA_C,SA_P,SA_NU,SA_NG,NER_pers,NER_norp,NER_fac,NER_org,NER_gpe,NER_loc,NER_prod,NER_eve,NER_woa,NER_law,NER_lang,NER_dat,NER_tim,NER_per,NER_mon,NER_quan,NER_ordi,NER_cardi
0,0,0,1,0,0,1,1,0,0,6,0,0,0,0,0,1,0,0,2,0,0.2500,0.192,0.692,0.115,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0
1,0,0,1,0,0,3,1,0,0,9,0,0,1,0,1,4,1,0,5,2,0.3612,0.098,0.902,0.000,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,2,0,0,4,0,0,0,9,0,0,0,0,0,3,1,0,3,0,0.3182,0.206,0.687,0.107,3,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,2,0,0,7,0,0,0,0,0,1,1,0,2,0,0.7579,0.394,0.606,0.000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,2,0,0,2,1,0,0,3,0,0,1,0,0,1,0,0,1,0,0.0000,0.000,1.000,0.000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10237,0,0,1,2,0,4,1,0,0,7,0,0,0,0,0,1,0,0,2,0,-0.7506,0.062,0.619,0.319,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
10238,1,0,2,0,0,2,0,0,0,8,0,0,0,1,0,1,0,0,2,0,0.4019,0.172,0.828,0.000,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
10239,0,0,2,0,0,5,2,0,1,11,0,0,0,0,0,3,1,0,5,1,0.5859,0.161,0.839,0.000,0,0,0,2,2,0,0,0,0,0,0,0,0,0,0,0,0,0
10240,1,0,1,0,0,1,0,0,0,5,0,0,0,0,0,1,1,0,2,0,0.0000,0.000,1.000,0.000,0,1,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0


## Train model

In [ ]:
bcs.fit(X_train, y_train, X_test, y_test)

Classifier = Nearest Neighbors, Score (test, accuracy) = 20.68, Training time = 0.56 seconds
Classifier = Linear SVM, Score (test, accuracy) = 25.49, Training time = 58.24 seconds
Classifier = RBF SVM, Score (test, accuracy) = 21.15, Training time = 119.31 seconds
Classifier = Decision Tree, Score (test, accuracy) = 23.76, Training time = 0.03 seconds
Classifier = Random Forest, Score (test, accuracy) = 24.55, Training time = 0.04 seconds
Classifier = Neural Net, Score (test, accuracy) = 24.23, Training time = 14.78 seconds
Classifier = AdaBoost, Score (test, accuracy) = 24.23, Training time = 0.74 seconds
Classifier = Naive Bayes, Score (test, accuracy) = 8.52, Training time = 0.01 seconds
Classifier = QDA, Score (test, accuracy) = 7.58, Training time = 0.08 seconds
--------------------------------------------------------------------------------
Best --> Classifier = Linear SVM, Score (test, accuracy) = 25.49


/usr/local/lib/python3.6/dist-packages/sklearn/discriminant_analysis.py:691: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


## Export the trained model

In [ ]:
bcs.export('/content/')

## Inferencing

In [ ]:
t = bcs.predict(X_test)

In [ ]:
t

,Label
0,false
1,false
2,false
3,false
4,half-true
...,...
1262,false
1263,false
1264,false
1265,false


## Generate possibilities

In [ ]:
prob_test = bcs.clf.predict_proba(bcs.extract(X_test))

In [ ]:
prob_test

array([[0.08203474, 0.21894051, 0.14794616, 0.18739633, 0.19645298,
        0.16722929],
       [0.08208278, 0.21521544, 0.14954956, 0.1961557 , 0.18587003,
        0.1711265 ],
       [0.08246543, 0.23984141, 0.18759866, 0.20146431, 0.14983595,
        0.13879424],
       ...,
       [0.08235875, 0.19065644, 0.18281095, 0.20882673, 0.18085837,
        0.15448877],
       [0.08235624, 0.20778048, 0.18379886, 0.20417851, 0.16247084,
        0.15941508],
       [0.08173541, 0.1952643 , 0.16854824, 0.20582555, 0.20163806,
        0.14698844]])

In [ ]:
bcs.clf.classes_

array([0, 1, 2, 3, 4, 5])

# ENCODING

In [ ]:
import keras
from keras import layers

encoding_dim = 16

input = keras.Input(shape=(42,))
encoded = layers.Dense(encoding_dim, activation='relu')(input)
decoded = layers.Dense(42, activation='sigmoid')(encoded)

autoencoder = keras.Model(input, decoded)

encoder = keras.Model(input, encoded)

encoded_input = keras.Input(shape=(encoding_dim,))
decoder_layer = autoencoder.layers[-1]
decoder = keras.Model(encoded_input, decoder_layer(encoded_input))

autoencoder.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

In [ ]:
p_X_train = bcs.extract(X_train)
p_X_test = bcs.extract(X_test)

In [ ]:
X_test

,1,Statement,4,5,6,7,8,9,10,11,12,13,14,15
0,11972.json,Building a wall on the U.S.-Mexico border will...,immigration,rick-perry,Governor,Texas,republican,30,30,42,23,18,Radio interview,"Meantime, engineering experts agree the wall w..."
1,11685.json,Wisconsin is on pace to double the number of l...,jobs,katrina-shankland,State representative,Wisconsin,democrat,2,1,0,0,0,a news conference,She cited layoff notices received by the state...
2,11096.json,Says John McCain has done nothing to help the ...,"military,veterans,voting-record",donald-trump,President-Elect,New York,republican,63,114,51,37,61,comments on ABC's This Week.,"Trump said that McCain ""has done nothing to he..."
3,5209.json,Suzanne Bonamici supports a plan that will cut...,"medicare,message-machine-2012,campaign-adverti...",rob-cornilles,consultant,Oregon,republican,1,1,3,1,1,a radio show,"But spending still goes up. In addition, many ..."
4,9524.json,When asked by a reporter whether hes at the ce...,"campaign-finance,legal-issues,campaign-adverti...",state-democratic-party-wisconsin,NaN,Wisconsin,democrat,5,7,2,2,7,a web video,Our rating A Democratic Party web video making...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1262,7334.json,Says his budget provides the highest state fun...,education,rick-scott,Governor,Florida,republican,28,23,38,34,7,a news conference,LeMieux didn't compare Rubio and Obama on an i...
1263,9788.json,Ive been here almost every day.,"civil-rights,crime,criminal-justice",jay-nixon,Governor,Missouri,democrat,2,0,0,1,0,"on ABC's ""This Week""","After making his pledge, Obama said the budget..."
1264,10710.json,"In the early 1980s, Sen. Edward Kennedy secret...","bipartisanship,congress,foreign-policy,history",mackubin-thomas-owens,"senior fellow, Foreign Policy Research Institute",Rhode Island,columnist,1,0,0,0,0,a commentary in The Providence Journal,Former President Clinton said government got t...
1265,3186.json,Says an EPA permit languished under Strickland...,"environment,government-efficiency",john-kasich,"Governor of Ohio as of Jan. 10, 2011",Ohio,republican,9,8,10,18,3,a news conference,Points of Light has a unique mission carved ou...


In [ ]:
autoencoder.fit(p_X_train, p_X_train,
                epochs=85,
                batch_size=256,
                shuffle=True,
                validation_data=(p_X_test, p_X_test))

Epoch 1/85
40/40 [==============================] - 0s 6ms/step - loss: 2.2492 - accuracy: 0.0000e+00 - val_loss: 2.9188 - val_accuracy: 0.0000e+00
Epoch 2/85
40/40 [==============================] - 0s 2ms/step - loss: 1.9745 - accuracy: 0.0312 - val_loss: 2.6597 - val_accuracy: 0.1507
Epoch 3/85
40/40 [==============================] - 0s 2ms/step - loss: 1.8047 - accuracy: 0.3887 - val_loss: 2.5724 - val_accuracy: 0.5549
Epoch 4/85
40/40 [==============================] - 0s 2ms/step - loss: 1.7597 - accuracy: 0.6249 - val_loss: 2.5540 - val_accuracy: 0.6717
Epoch 5/85
40/40 [==============================] - 0s 2ms/step - loss: 1.7482 - accuracy: 0.6974 - val_loss: 2.5461 - val_accuracy: 0.7001
Epoch 6/85
40/40 [==============================] - 0s 2ms/step - loss: 1.7417 - accuracy: 0.7250 - val_loss: 2.5399 - val_accuracy: 0.7309
Epoch 7/85
40/40 [==============================] - 0s 2ms/step - loss: 1.7363 - accuracy: 0.7274 - val_loss: 2.5348 - val_accuracy: 0.7395
Epoch 8/85
4

## Test encoder

In [ ]:
p_X_test

,CC,CD,DT,EX,FW,IN,JJ,LS,MD,NN,PDT,POS,PRP,RB,RP,SYM,TO,UH,VB,WH,SA_C,SA_P,SA_NU,SA_NG,NER_pers,NER_norp,NER_fac,NER_org,NER_gpe,NER_loc,NER_prod,NER_eve,NER_woa,NER_law,NER_lang,NER_dat,NER_tim,NER_per,NER_mon,NER_quan,NER_ordi,NER_cardi
0,0,0,2,0,0,1,0,0,1,4,0,0,0,1,0,1,0,0,2,0,0.0000,0.000,1.000,0.000,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
1,0,0,2,0,0,2,0,0,0,5,0,0,0,0,0,1,1,0,2,0,0.0772,0.106,0.894,0.000,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0
2,0,0,1,0,0,0,0,0,0,4,0,0,0,0,0,1,1,0,4,0,-0.3089,0.000,0.799,0.201,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,1,0,0,1,0,0,1,7,0,0,0,0,0,1,0,0,2,1,0.3400,0.288,0.577,0.135,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,3,0,0,4,1,0,0,9,0,0,0,0,0,3,1,1,3,1,-0.5994,0.099,0.659,0.242,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1262,0,0,1,0,0,2,1,0,0,6,0,0,1,0,0,1,0,0,2,0,0.0000,0.000,1.000,0.000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1263,0,0,1,0,0,0,0,0,0,2,0,0,0,2,0,1,0,0,1,0,0.0000,0.000,1.000,0.000,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
1264,0,1,2,0,0,2,3,0,0,8,0,0,0,1,0,2,1,0,3,0,0.4019,0.119,0.881,0.000,2,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
1265,1,1,1,0,0,2,1,0,0,6,0,0,2,0,0,1,0,0,4,0,0.0000,0.000,1.000,0.000,1,0,0,2,0,0,0,0,0,0,0,1,0,0,0,0,0,0


In [ ]:
encoder.predict(p_X_test)

array([[1.6610156, 4.9395194, 2.9832437, ..., 2.9553337, 4.8539658,
        2.2865112],
       [3.7399297, 3.4288416, 0.7154356, ..., 2.840631 , 4.7466674,
        2.065889 ],
       [3.1967797, 3.2853818, 1.986229 , ..., 4.155172 , 3.043211 ,
        2.496645 ],
       ...,
       [5.5865054, 1.2594497, 7.5452027, ..., 3.5307326, 8.895622 ,
        4.6662755],
       [5.7518034, 2.6379714, 4.580003 , ..., 4.2373204, 1.4538329,
        5.756905 ],
       [6.453235 , 3.27547  , 5.0840454, ..., 4.417034 , 5.9056306,
        5.0748243]], dtype=float32)

## Save encoder

In [ ]:
encoder.save('/content/bcs_encoder')

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: /content/bcs_encoder/assets


## Load and test encoder

In [ ]:
encoder_reloaded = keras.models.load_model('/content/bcs_encoder')

In [ ]:
encoder_reloaded.predict(p_X_test)

array([[14.840651 ,  4.7654595, 14.986643 , ..., 12.210411 , 14.633874 ,
         9.988829 ],
       [15.701    ,  4.0937634, 12.443446 , ...,  9.242275 ,  5.9453783,
        16.075352 ],
       [ 9.099114 ,  9.711052 , 12.322859 , ...,  9.27873  ,  7.040361 ,
        10.212723 ],
       ...,
       [14.428387 , 15.557939 , 21.380577 , ..., 19.371414 , 10.255583 ,
        13.62308  ],
       [14.294861 , 11.599106 ,  9.861072 , ..., 20.696312 , 16.158012 ,
        11.004672 ],
       [10.039318 , 23.554214 , 16.21031  , ...,  6.7429595, 16.444586 ,
        15.835458 ]], dtype=float32)

## Zip encoder

In [ ]:
import os
import zipfile

def zipdir(path, ziph):
    # ziph is zipfile handle
    for root, dirs, files in os.walk(path):
        for file in files:
            ziph.write(os.path.join(root, file))

In [ ]:
%cd /content/
zipf = zipfile.ZipFile('bcs_encoder.zip', 'w', zipfile.ZIP_DEFLATED)
zipdir('bcs_encoder/', zipf)
zipf.close()

/content


## Unzip encoder

In [ ]:
archive = ZipFile('bcs_encoder.zip')

for file in archive.namelist():
    archive.extract(file, '/content/test/')

## Test encoder in the module

In [ ]:
bcs.encode(X_test)

array([[14.840651 ,  4.7654595, 14.986643 , ..., 12.210411 , 14.633874 ,
         9.988829 ],
       [15.701    ,  4.0937634, 12.443446 , ...,  9.242275 ,  5.9453783,
        16.075352 ],
       [ 9.099114 ,  9.711052 , 12.322859 , ...,  9.27873  ,  7.040361 ,
        10.212723 ],
       ...,
       [14.428387 , 15.557939 , 21.380577 , ..., 19.371414 , 10.255583 ,
        13.62308  ],
       [14.294861 , 11.599106 ,  9.861072 , ..., 20.696312 , 16.158012 ,
        11.004672 ],
       [10.039318 , 23.55421  , 16.21031  , ...,  6.74296  , 16.444586 ,
        15.835457 ]], dtype=float32)